In [1]:
import numpy as np
import pandas as pd
from graspy.utils import * 
from graspy.plot import heatmap, pairplot, gridplot
from graspy.embed import *
import seaborn as sns
import matplotlib.pyplot as plt
import re
import scipy
import networkx as nx
from pathlib import Path
from sklearn.utils.graph_shortest_path import graph_shortest_path
%matplotlib inline

In [3]:
basedir = Path('../../../eleGraSPy')
basedir = basedir / 'Cook et al revised Supplementary Information'
matdir = basedir / 'SI 5 Connectome adjacency matrices Pedigo.xlsx'
celldir = basedir / 'herm_cell_list_pedigo.xlsx'
si3_path = basedir / 'SI 3 Synapse lists.xlsx'

In [4]:
herm = pd.read_excel(si3_path, sheet_name=0)
male = pd.read_excel(si3_path, sheet_name=1)
new_labels = ['num_contin', 'em_series', 'pre', 'all_post', 'type', 'sections',
 'num_post', 'post1', 'post2', 'post3', 'post4']
herm.columns = new_labels
male.columns = new_labels
herm_chem = herm.loc[herm['type'] == 'chemical']
herm_elec = herm.loc[herm['type'] == 'electrical']
male_chem = male.loc[male['type'] == 'chemical']
male_elec = male.loc[male['type'] == 'electrical']

In [5]:
def get_pre_post(row):
    pre = row.pre
    post_names = re.split('[, .]', row.all_post)
    pre = re.sub('[\[\] \\-#]', '', pre)
    post_names = [re.sub('[\[\] \\-#]', '', post) for post in post_names]
    return pre, post_names

def get_cells(df):
    names = set()
    for i, row in df.iterrows():
        pre, post_names = get_pre_post(row)
        names.add(pre)
        [names.add(post) for post in post_names]
    if '' in names:
        names.remove('')
    cell_ids = np.unique(list(names))
    inds = range(len(cell_ids))
    cell_id_map = dict(zip(cell_ids,inds))
    return cell_ids, cell_id_map

def df_to_adjacency(df, nodelist, weight=1, directed=True):
    rows = []
    for i, row in df.iterrows():
        pre, post_names = get_pre_post(row)
        for a in post_names:
            # pre | post | weight
            rows.append([pre, a, weight])
    
    df_split = pd.DataFrame(rows, columns=['source', 'target', 'weight'])
    if directed:
        g = nx.MultiDiGraph()
    else:
        g = nx.MultiGraph()
    g = nx.from_pandas_edgelist(df_split, create_using=g)
    return nx.to_numpy_array(g, nodelist=nodelist)

In [6]:
herm_cells, herm_cell_map = get_cells(herm_chem)
male_cells, male_cell_map = get_cells(male_chem)